# Regressão linear

Regressão é a tarefa de encontrar uma função que aproxima um conjunto de dados, de forma que a variável objetivo é contínua (ou seja, a função tem como imagem $\mathbb{R}$). Neste exemplo, trataremos de um conjunto de dados onde a variável objetivo é o preço mediano das casas em um determinado bairro, e temos acesso a uma única variável preditora que é a porcentagem da população daquele bairro que é considerada de baixa renda.

Intuitivamente, sabemos que bairros com maior concentração de pessoas de baixa renda tem algum tipo de relação com um menor preço das casas naquele bairro.

Começaremos olhando para os nossos dados para ter uma ideia geral do problema que estamos lidando. A biblioteca `pandas` nos dá acesso a funções de tratamento de dados tabulares. Utilizamos essa biblioteca para importar nossos dados, olhar algumas estatísticas básicas e visualizar os dados.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.datasets import load_boston

%matplotlib inline

In [ ]:
boston = load_boston()

In [ ]:
dados = pd.DataFrame({'baixa renda': boston.data[:, -1],
                      'preço': boston.target})
dados.head()

In [ ]:
dados = dados.sort_values(by='baixa renda')
dados.head()

In [ ]:
dados.shape

In [ ]:
dados.describe()

In [ ]:
dados.plot.scatter('baixa renda', 'preço');

In [ ]:
dados.hist();

In [ ]:
dados.hist(bins=5);

## Coeficiente de correlação

O coeficiente de correlação de Pearson, muitas vezes chamado somente de correlação ou de Pearson r, é uma estatística que calcula o quanto uma variável é linearmente dependente de outra. O valor varia entre -1 (indicando uma correlação perfeita negativa, ou seja, o aumento da variável independente causa uma redução na variável dependente) e 1 (similarmente uma correlação perfeita, mas positiva). O valor zero indica que não há correlação linear entre as variáveis.

Existem outros tipos de correlação para outros casos, mas o coeficiente de Pearson é utilizado para dependências lineares.

Em Python, temos a função de cálculo de correlação `pearsonr` dentro do pacote `scipy.stats`. A função retorna um par de números, sendo que o primeiro é o coeficiente de correlação (o segundo número é o p-valor, que ignoraremos).

In [ ]:
pearsonr(dados['baixa renda'], dados['preço'])[0]

In [ ]:
renda = dados['baixa renda'].values.reshape(-1, 1)
preco = dados['preço'].values.reshape(-1, 1)

## Algoritmos de regressão

Como estamos interessados em regressão linear, os algoritmos utilizados retornarão uma reta, geralmente representada pela equação $y = mx + y_0$, onde estamos interessados em encontrar os valores $m$ e $y_0$. Na biblioteca `sklearn`, esses valores são chamados de `coef_` e `intercept_`.

## Regressão linear simples

In [ ]:
regressor = LinearRegression()
regressor.fit(renda, preco)
y0 = regressor.intercept_
m = regressor.coef_[0]
y0, m

In [ ]:
plt.scatter(renda, preco, marker='.')

x = np.linspace(0, 40, 2).reshape(-1, 1)
plt.plot(x, regressor.predict(x), c='r')

plt.axvline(0, c='gray')
f0 = regressor.predict(np.array([0]).reshape(-1, 1))
plt.axhline(f0, c='gray')

f0

### Avaliando uma regressão

Existem diversos cálculos para avaliar a qualidade de uma regressão. Comumente utilizados temos o fator $R^2$, a média dos erros absolutos, e a média dos erros quadrados. Todas essas funções estão disponíveis no módulo `sklearn.metrics`.

O fator $R^2$, chamado de coeficiente de determinação, pode ser visto como quanto uma variável tem poder de predição sobre a outra. Para o caso da regressão linear, o $R^2$ é equivalente ao quadrado do coeficiente de Pearson.

Utilizaremos a função `predict` do nosso regressor para encontrar os valores de `y` da reta nos pontos específicos onde há exemplos.

In [ ]:
predito = regressor.predict(renda)
r2_score(preco, predito)

In [ ]:
pearsonr(preco, renda)[0] ** 2

In [ ]:
mean_absolute_error(preco, predito)

In [ ]:
mean_squared_error(preco, predito)

## Regressão linear com penalidade L1 (Lasso)

Outros algoritmo comuns para regressão são Lasso e Ridge, que utilizam as penalidades L1 e L2, respectivamente, para regularizar os dados. Também existe o Elastic Net, que combina as duas penalidades.

Esse tipo de regularização fazem com que valores muito fora do comum (_outliers_) sejam menos considerados, e os coeficientes tendem a ficar mais simples. Essa regularização é muito comum para quando há mais de uma variável preditora, mas utilizaremos aqui para mostrar o funcionamento.

O algoritmo Lasso utiliza um hiper-parâmetro $\alpha$ (alfa), com valor padrão 1.0, que determina quanto de penalidade deve ser aplicado. Quando esse valor tende a zero, o algoritmo se aproxima do algoritmo de regressão simples (mas não é recomendado fazer isso pois há instabilidade numérica).

In [ ]:
regressor = Lasso(alpha=1.0).fit(renda, preco)
y0 = regressor.intercept_
m = regressor.coef_
print(y0, m)

x = np.linspace(0, 40, 2).reshape(-1, 1)
plt.scatter(renda, preco, marker='.')
plt.plot(x, regressor.predict(x), color='red');

In [ ]:
predito = regressor.predict(renda)
print('R^2', r2_score(preco, predito))
print('MAE', mean_absolute_error(preco, predito))
print('MSE', mean_squared_error(preco, predito))

Como podemos ver, o coeficiente mudou muito pouco, de -0.95 para -0.93. Os valores de erro também continuam praticamente iguais. Tentemos agora com valores diferentes de $\alpha$.

In [ ]:
cores = ['red', 'green', 'black', 'gray']
alfas = [0.1, 0.5, 0.8, 1.5]

plt.scatter(renda, preco, marker='.')

for cor, alfa in zip(cores, alfas):
    regressor = Lasso(alpha=alfa).fit(renda, preco)
    y0 = regressor.intercept_
    m = regressor.coef_[0]
    
    predito = regressor.predict(renda)
    r2 = r2_score(preco, predito)
    mae = mean_absolute_error(preco, predito)
    mse = mean_squared_error(preco, predito)
    print(f'''
    y0: {y0}
    m: {m}
    R^2: {r2}
    Erro absoluto: {mae}
    Erro quadrado: {mse}
    ''')
    
    plt.plot(x, regressor.predict(x), color=cor)

Alterações em $\alpha$ causam poucas variações da reta gerada.

É importante notar que as avaliações de algoritmos de aprendizado geralmente não são aplicadas no mesmo conjunto de dados utilizado para treinar o algoritmo, como estamos fazendo aqui. Mais para frente utilizaremos avaliações menos ingênuas.

## Alteração de atributos / atributos não-lineares

Um problema comum em ciência de dados é encontrar os atributos corretos. Muitas vezes, podemos aplicar funções matemáticas para alterar ou combinar os atributos que temos em mãos para encontrar modelos superiores.

Podemos pensar, existe algum outro tipo de função que se adequaria melhor aos nossos dados do que uma reta? Para quem está acostumado com a função $\log$, a distribuição dos nossos dados se assemelha um pouco com a função $\log$ aplicada no inverso do número, como podemos visualizar abaixo.

In [ ]:
x = np.linspace(3, 100, 100)
y = np.log(1 / x)
plt.plot(x, y);

Como podemos fazer para ajustar uma curva logarítmica aos nossos dados? A resposta é que não precisamos! Basta alterar os atributos para que estejam em espaço log, e então aplicar uma regressão linear.

In [ ]:
plt.scatter(np.log(renda), preco, marker='.');

In [ ]:
pearsonr(np.log(renda), preco)[0]

Como podemos ver, o coeficiente de correlação foi de -0.737 para -0.815, indicando que agora há uma relação _linear_ mais forte entre as variáveis.

In [ ]:
cores = ['red', 'green', 'black', 'gray', 'yellow']
alfas = [0.1, 0.5, 0.8, 1.0, 1.5]

plt.scatter(np.log(renda), preco, marker='.')

for cor, alfa in zip(cores, alfas):
    regressor = Lasso(alpha=alfa).fit(np.log(renda), preco)
    y0 = regressor.intercept_
    m = regressor.coef_[0]
    
    predito = regressor.predict(np.log(renda))
    r2 = r2_score(preco, predito)
    mae = mean_absolute_error(preco, predito)
    mse = mean_squared_error(preco, predito)
    print(f'''
    y0: {y0}
    m: {m}
    R^2: {r2}
    Erro absoluto: {mae}
    Erro quadrado: {mse}
    ''')
    
    x = np.linspace(0, np.log(40), 200).reshape(-1, 1)
    plt.plot(x, regressor.predict(x), color=cor)

In [ ]:
regressor = LinearRegression().fit(np.log(renda), preco)
x = np.linspace(renda.min(), renda.max(), 300).reshape(-1, 1)
y = regressor.predict(np.log(x))

plt.scatter(renda, preco, marker='.')
plt.plot(x, y, c='r');

## Regressão polinomial

Contraintuitivamente, regressão polinomial é só um caso específico de regressão linear. A regressão linear pode ser, e geralmente é feita, utilizando mais de uma variável preditora, permitindo modelos mais complexos. Nesse caso, a equação da reta em múltiplas dimensões passa a ser $y = y_0 + m_0x_0 + m_1x_1 + m_2x_2 + \ldots + m_nx_n$. Os valores de $x$ são os valores observados das variáveis; o algoritmo de regressão é responsável por achar o valor $y_0$ e todos os valores dos coeficientes $m$.

Consideremos agora que temos somente uma variável preditora, como no conjunto de dados que estamos tratando. Podemos modificar a variável preditora como fizemos com a operação $\log$, gerando várias variáveis que podem ser usadas em uma regressão linear multidimensional, como acima. No caso específico que modificamos nossa variável preditora $x$ utilizando suas potências, ficamos com as variáveis preditoras $x, x^2, x^3, ..., x^n$, para um determinado $n$ que escolhermos. Assim, a equação da "reta" ajustada passa a ser $y = y_0 + m_0x + m_1x^2 + m_2x^3 + ... + m_nx^{n+1}$, um polinômio de grau $n+1$.

Vamos começar ajustando um polinômio de grau 2 (uma parábola) aos nossos dados, e comparar com um polinômio de grau 14.

Trivia: o nome regressão linear se refere a uma combinação linear entre as variáveis, e não porque se ajusta uma linha. Por isso a regressão polinomial é uma regressão linear. A equação da regressão polinomial no primeiro parágrafo dessa célula também pode ser escrita como $y - y_0 = \vec{m} \cdot \vec{x} = \sum m_i x_i$, onde o lado direito da equação é a forma canônica da combinação linear. Mesmo que as variáveis sejam alteradas por diversas funções não-lineares, como fizemos com $\log$ anteriormente, a combinação entre os atributos é sempre feita nessa forma linear, multiplicando os coeficientes pelos valores de atributos, e somando o resultado. No caso polinomial, temos $x_i = x^i$.

In [ ]:
x1 = renda
x2 = renda ** 2
x = np.hstack([x1, x2])
regressor = LinearRegression().fit(x, preco)
y0 = regressor.intercept_
m = regressor.coef_
print(y0, m)

curva = regressor.predict(x)
print(r2_score(preco, curva), mean_absolute_error(preco, curva), mean_squared_error(preco, curva))

plt.scatter(renda, preco)
plt.plot(renda, curva, c='red');

In [ ]:
x_l = [renda]
for potencia in range(2, 15):
    x_l.append(renda ** potencia)

x = np.hstack(x_l)
regressor = LinearRegression().fit(x, preco)
print(regressor.coef_)
curva = regressor.predict(x)
print(r2_score(preco, curva), mean_absolute_error(preco, curva), mean_squared_error(preco, curva))
plt.scatter(renda, preco)
plt.plot(renda, curva, c='red');

Apesar do polinômio de grau 14 ser um modelo mais complexo, seu erro é maior que os polinômios de grau 1 (reta) e 2. Isso acontece porque o modelo está _superajustado_ (_overfitted_) aos dados representados, e não generaliza a tendência dos dados. Nesses casos, podemos utilizar as penalidades L1 e L2 como discutidos anteriormente.

(Pode ser que um aviso de `ConvergenceWarning` seja mostrado; não é um erro do programa, mas sim um aviso de que o resultado pode ser numericamente instável devido à alta dimensão).

In [ ]:
cores = ['red', 'green', 'black', 'gray', 'yellow']
alfas = [0.1, 0.5, 0.8, 1.0, 1.5]

plt.scatter(renda, preco)

for cor, alfa in zip(cores, alfas):
    regressor = Lasso(alpha=alfa)
    regressor.fit(x, preco)
    
    predito = regressor.predict(x)
    print(r2_score(preco, predito), mean_squared_error(preco, predito))
    
    graphx = np.linspace(0, np.log(40), 200).reshape(-1, 1)
    plt.plot(renda, predito, color=cor)

Apesar da curva gerada ser um polinômio de grau 14, sua aparência é muito similar à parabola que geramos anteriormente. Dessa forma, o modelo consegue representar curvas mais complexas, mas tem uma tendência a utilizar formas mais simples, generalizando a tendência dos dados.